In [1]:
#DATA CLEANING

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#Read csv file also set its data type
df = pd.read_csv(
    'eco.csv',
    encoding='latin1',
    dtype={
        'CustomerID': str,
        'StockCode': str,
        'InvoiceNo': str,
        'Quantity': int,
        'UnitPrice': float
    }
)

#Change InvoiceDate to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

#remove duplicates
df=df.drop_duplicates()

#trip spaces in all string columns

str_cols = df.select_dtypes(include='object').columns
df[str_cols] = df[str_cols].apply(lambda x: x.str.strip())


# Drop rows where CustomerID is NaN or blank (only spaces)
df = df[df['CustomerID'].notna() & (df['CustomerID'].str.strip() != '')]

# negative quantity
returns_df = df[df["Quantity"] < 0]

#Handle negative or zero quantities/prices if needed

df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

#To add Purchase Value column by multiplying unit price and quantity
df['Purchase_Value']=df['UnitPrice']*df['Quantity']
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Purchase_Value
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,16.60


In [2]:
# First, get the first and last purchase dates per customer
df
# Average Purchase Value & Average Purchase Frequency

#Formula
# Avg Purchase Value = Total Purchase Value/Total No of purchase
#Avg Purchase freq = Total No of purchase / No of unique customers

Total_Purchase_value = df['Purchase_Value'].sum()
Total_No_of_purchase = df['InvoiceNo'].nunique()
No_of_unique_customers = df['CustomerID'].nunique()

avg_purchase_value = Total_Purchase_value/Total_No_of_purchase
avg_purchase_freq = Total_No_of_purchase / No_of_unique_customers



lifespans = df.groupby('CustomerID').agg(
    first_purchase=('InvoiceDate', 'min'),
    last_purchase=('InvoiceDate', 'max')
)

lifespans['lifespan_in_days'] = (lifespans['last_purchase'] - lifespans['first_purchase']).dt.days
avg_lifespan_in_months = lifespans['lifespan_in_days'].mean()/30

# Given values
gross_margin = 0.3
CAC = 5000

# CLV calculation
clv = (avg_purchase_value * gross_margin * avg_purchase_freq * avg_lifespan_in_months) - CAC

print("Average Purchase Value:", avg_purchase_value)
print("Purchase Frequency:", avg_purchase_freq)
print("Average Lifespan (months):", avg_lifespan_in_months)
print("CLV:", clv)


Average Purchase Value: 479.56016047917115
Purchase Frequency: 4.272014753342554
Average Lifespan (months): 4.3482864607345935
CLV: -2327.5152069501896
